In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd /
from glob import glob

check_img_list = glob('/content/gdrive/MyDrive/Colab_Notebooks/dataset3/export/images/check/*.jpg') + glob('/content/gdrive/MyDrive/Colab Notebooks/dataset2/export/images/check*.JPG')
pass_img_list = glob('/content/gdrive/MyDrive/Colab_Notebooks/dataset3/export/images/pass/*.jpg') + glob('/content/gdrive/MyDrive/Colab Notebooks/dataset2/export/images/pass*.JPG')
rot_img_list = glob('/content/gdrive/MyDrive/Colab_Notebooks/dataset3/export/images/rot/*.jpg') + glob('/content/gdrive/MyDrive/Colab Notebooks/dataset2/export/images/rot*.JPG')

print(len(check_img_list + pass_img_list + rot_img_list))

/
2295


In [ ]:
from sklearn.model_selection import train_test_split

train_check_img_list, val_check_img_list = train_test_split(check_img_list, test_size=0.2, random_state=2000)
train_pass_img_list, val_pass_img_list = train_test_split(pass_img_list, test_size=0.2, random_state=2000)
train_rot_img_list, val_rot_img_list = train_test_split(rot_img_list, test_size=0.2, random_state=2000)

train_img_list = train_check_img_list + train_pass_img_list + train_rot_img_list
val_img_list = val_check_img_list + val_pass_img_list + val_rot_img_list

print(len(train_img_list), len(val_img_list))

1835 460


In [ ]:
with open('/content/gdrive/MyDrive/Colab_Notebooks/dataset3/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/gdrive/MyDrive/Colab_Notebooks/dataset3/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

with open('/content/gdrive/MyDrive/Colab_Notebooks/dataset3/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/gdrive/MyDrive/Colab_Notebooks/dataset3/train.txt'
data['val'] = '/content/gdrive/MyDrive/Colab_Notebooks/dataset3/val.txt'

with open('/content/gdrive/MyDrive/Colab_Notebooks/dataset3/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 3, 'names': ['check', 'pass', 'rot']}
{'train': '/content/gdrive/MyDrive/Colab_Notebooks/dataset3/train.txt', 'val': '/content/gdrive/MyDrive/Colab_Notebooks/dataset3/val.txt', 'nc': 3, 'names': ['check', 'pass', 'rot']}


In [ ]:
%cd /content/yolov5/

!python train.py --img 224 --batch 1 --epochs 100 --data /content/gdrive/MyDrive/Colab_Notebooks/dataset3/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name tomato_yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/gdrive/MyDrive/Colab_Notebooks/dataset3/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=1, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=tomato_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-355-ge5991c9 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_

In [ ]:
from models.yolo import Model
import torch
device = torch.device("cpu")
# yaml_path='./dataset/data.yaml'
yaml_path = './content/gdrive/MyDrive/Colab_Notebooks/dataset3/data.yaml'
new_weights= '/content/yolov5/runs/train/tomato_yolov5s_results/weights/best.pt'
model = Model(yaml_path).to(device)
trained_model = torch.load(new_weights,map_location=device)['model'].to(device)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

In [ ]:
from IPython.display import Image
import os

val_img_path = val_img_list[100]

!python detect.py --weights /content/yolov5/runs/train/tomato_yolov5s_results/weights/best.pt --img 224 --conf 0.5 --source "{val_img_path}"
Image(os.path.join('/content/yolov5/inference/output', os.path.basename(val_img_path)))

In [ ]:
model_save_name = 'yolo_fruit3.pt'
path = F"/content/gdrive/My Drive/Colab_Notebooks/{model_save_name}" 
torch.save(model.state_dict(), path)
print(f'{model_save_name} saved successfully.')

In [ ]:
!tensorboard dev upload --logdir runs \